<a href="https://colab.research.google.com/github/darsigov92/assistent/blob/main/PerfAssist_Complete_Gradio_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка библиотек
!pip install gspread oauth2client gradio scikit-learn pandas --quiet

In [ ]:
# Импорты и подключение к Google Таблице
import gradio as gr
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sklearn.linear_model import LogisticRegression
import numpy as np

# Настройка доступа
SHEET_NAME = "patient_database"
WORKSHEET_NAME = "Лист1"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("your-credentials.json", scope)
client = gspread.authorize(creds)
sheet = client.open(SHEET_NAME).worksheet(WORKSHEET_NAME)

# Получение и сохранение данных
def get_data():
    # Ensure we get header and all data
    data = sheet.get_all_values()
    if not data:
        return pd.DataFrame()
    headers = data[0]
    return pd.DataFrame(data[1:], columns=headers)

sheet_columns = sheet.row_values(1)
def save_patient(data_dict):
    df = get_data()
    patient_id = data_dict.get("ID пациента", "").strip() # Get ID and remove leading/trailing whitespace

    if patient_id and patient_id in df["ID пациента"].astype(str).values:
        # Find the row index (gspread is 1-indexed, header is row 1)
        # .astype(str) is used to handle potential type mismatches between input and sheet data
        try:
            row_index = df[df["ID пациента"].astype(str) == patient_id].index[0] + 2
            values = [str(data_dict.get(col, "")) for col in sheet_columns]
            sheet.update_row(row_index, values)
            return f"Пациент с ID {patient_id} обновлён."
        except Exception as e:
            return f"Ошибка при обновлении пациента: {e}"

    else:
        try:
            values = [str(data_dict.get(col, "")) for col in sheet_columns]
            sheet.append_row(values)
            return "Новый пациент сохранён."
        except Exception as e:
            return f"Ошибка при сохранении нового пациента: {e}"


# Обучение и прогноз
model = None
X_columns = ["Возраст", "ФВ ЛЖ", "ИМТ", "СД", "Креатинин", "СКФ", "EuroSCORE II"]
def train_model():
    global model
    df = get_data().dropna(subset=X_columns + ["Исход"]) # Drop rows with missing training data
    if len(df) < 10:
        return "Недостаточно данных для обучения (требуется минимум 10 полных записей)"
    X = df[X_columns]
    y = df["Исход"]
    # Ensure y is numeric
    y = pd.to_numeric(y, errors='coerce').dropna()
    X = X.loc[y.index] # Align X and y after dropping NaNs in y

    if len(X) < 10:
         return "Недостаточно данных для обучения после очистки"

    model = LogisticRegression()
    model.fit(X, y)
    return f"Модель обучена на {len(X)} пациентах."

def predict_outcome(age, ef, bmi, diabetes, creatinine, gfr, euroscore):
    if model is None:
        return "Сначала обучите модель."
    # Ensure input is in correct format and handle potential errors
    try:
        x = np.array([[float(age), float(ef), float(bmi), float(diabetes), float(creatinine), float(gfr), float(euroscore)]])
        prob = model.predict_proba(x)[0][1]
        return f"Вероятность неблагоприятного исхода: {round(prob * 100, 2)}%"
    except ValueError:
        return "Ошибка: Введите числовые значения для прогноза."
    except Exception as e:
        return f"Произошла ошибка при прогнозировании: {e}"

In [ ]:
# Интерфейс Gradio с блоками и всеми полями
with gr.Blocks() as demo:
    gr.Markdown("# Полный интерфейс для оценки риска у пациентов")
    with gr.Tab("Общие данные"):
        id = gr.Text(label="ID пациента")
        age = gr.Number(label="Возраст")
        dob = gr.Text(label="Дата рождения")
        sex = gr.Radio(["М", "Ж"], label="Пол")
        bmi = gr.Number(label="ИМТ")
        emergency = gr.Radio([1, 0], label="Экстренность (1=план, 0=экстр)")
        surgery_date = gr.Text(label="Дата операции")
        cpb_group = gr.Radio(["ИК", "Без ИК"], label="Группа")
        surgery_type = gr.Dropdown(["АКШ с ИК", "АКШ без ИК", "Клапанная", "АКШ + Клапанная", "Аорта", "Комбинированная", "Другое"], label="Тип операции")
        surgeon = gr.Text(label="Хирург")
        notes = gr.Textbox(label="Примечания")
    with gr.Tab("Кардиология"):
        ef = gr.Number(label="ФВ ЛЖ")
        af_type = gr.Radio(["ФП", "Пароксизмы", "Нет"], label="ФП / Пароксизмы")
        parox_count = gr.Number(label="Кол-во пароксизмов")
        angina_fc = gr.Radio(["I", "II", "III", "IV"], label="ФК ИБС")
        chf_stage = gr.Text(label="Стадия ХСН")
        chf_fc = gr.Radio(["1", "2", "3", "4"], label="ФК ХСН (NYHA)")
        cerebral_as = gr.Radio(["Да", "Нет"], label="Церебральный атеросклероз")
        bca_as = gr.Radio(["Да", "Нет"], label="Атеросклероз БЦА")
        leg_vessel_as = gr.Radio(["Да", "Нет"], label="Сосуды нижних конечностей")
        lv_aneurysm = gr.Radio(["Да", "Нет"], label="Аневризма ЛЖ")
    with gr.Tab("Лаборатория и почки"):
        nt_pro_bnp = gr.Number(label="Натрий-уретический пептид")
        syst_pa = gr.Text(label="СДЛА")
        edv = gr.Number(label="КДО")
        protein = gr.Number(label="Общий белок")
        urea = gr.Number(label="Мочевина")
        creatinine = gr.Number(label="Креатинин")
        ckd_stage = gr.Text(label="ХБП стадия")
        gfr = gr.Number(label="СКФ")
        hemoglobin = gr.Number(label="Гемоглобин")
    with gr.Tab("Поведение и исход"):
        diabetes = gr.Radio([1, 0], label="СД 2 типа")
        smoking = gr.Number(label="Курение (пачка/лет)")
        euroscore = gr.Number(label="EuroSCORE II")
        mini_access = gr.Radio(["Да", "Нет"], label="Мини-доступ")
        outcome = gr.Radio([1, 0], label="Исход (1=осложнение/смерть, 0=нет)")

    output = gr.Textbox(label="Результат")
    save = gr.Button("💾 Сохранить пациента")
    train = gr.Button("🧠 Обучить модель")
    predict = gr.Button("📈 Прогноз")

    inputs = [age, ef, bmi, diabetes, creatinine, gfr, euroscore]
    save.click(lambda *vals: save_patient(dict(zip(sheet_columns, vals))),
              inputs=[id, age, ef, cpb_group, dob, sex, bmi, emergency, surgery_date, diabetes, af_type, parox_count, angina_fc, chf_stage, chf_fc, cerebral_as, nt_pro_bnp, syst_pa, edv, protein, urea, creatinine, ckd_stage, gfr, hemoglobin, chf_fc, smoking, euroscore, surgeon, notes, leg_vessel_as, lv_aneurysm, mini_access, outcome],
              outputs=output)
    train.click(fn=train_model, outputs=output)
    predict.click(fn=predict_outcome, inputs=inputs, outputs=output)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://095e61400d3da42c5f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


После обучения модели, вы можете использовать её для прогнозирования. Например, вот как можно сделать предсказание для нового пациента с заданными параметрами.

In [20]:
# Пример использования обученной модели для прогнозирования
# Убедитесь, что модель обучена, прежде чем запускать этот код.
# Вызовите train_model() или нажмите кнопку "Обучить модель" в интерфейсе Gradio.

if model is not None:
    # Пример данных для нового пациента (в том же порядке, что и X_columns)
    # ["Возраст", "ФВ ЛЖ", "ИМТ", "СД", "Креатинин", "СКФ", "EuroSCORE II"]
    new_patient_data = np.array([[65, 45, 28, 1, 1.2, 75, 5.5]])

    # Прогнозирование вероятности неблагоприятного исхода
    predicted_probability = model.predict_proba(new_patient_data)[0][1]

    print(f"Прогнозируемая вероятность неблагоприятного исхода для нового пациента: {round(predicted_probability * 100, 2)}%")
else:
    print("Модель еще не обучена. Пожалуйста, сначала обучите модель.")

Модель еще не обучена. Пожалуйста, сначала обучите модель.
